<a href="https://colab.research.google.com/github/LuanNguyen-git/Template/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
# Tạo dữ liệu chuỗi thời gian mẫu
def create_dataset(data, look_back=1):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:(i + look_back)])
        y.append(data[i + look_back])
    return np.array(X), np.array(y)

In [ ]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Thay đổi đường dẫn đến file .xlsx của bạn tại đây
file_path = '/content/drive/MyDrive/Articles/Article 09/Code/MATLAB/CPI_VN.xlsx'

# Đọc file .xlsx vào DataFrame
try:
    df = pd.read_excel(file_path)
    print("Đã đọc file thành công!")
    display(df.head())
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file tại đường dẫn {file_path}. Vui lòng kiểm tra lại đường dẫn.")
except Exception as e:
    print(f"Đã xảy ra lỗi khi đọc file: {e}")

In [ ]:
cpi_data = df['cpi'].values
print("Dữ liệu CPI đã được trích xuất:")
display(cpi_data)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Chia dữ liệu thành tập train và test trước
train_size = int(len(cpi_data) * 0.67)
test_data = cpi_data[train_size:]
train_validation_data = cpi_data[:train_size]

# Chuẩn hóa dữ liệu tập train và validation
scaler = MinMaxScaler(feature_range=(0, 1))
train_validation_data_scaled = scaler.fit_transform(train_validation_data.reshape(-1, 1))
test_data_scaled = scaler.transform(test_data.reshape(-1, 1)) # Sử dụng cùng scaler để chuẩn hóa tập test

# Tách tập train_validation thành train và validation
validation_size = int(len(train_validation_data_scaled) * 0.15 / 0.67) # Tính toán lại kích thước validation dựa trên tập train_validation
train_data_scaled = train_validation_data_scaled[0:len(train_validation_data_scaled) - validation_size, :]
validation_data_scaled = train_validation_data_scaled[len(train_validation_data_scaled) - validation_size:, :]


# Tạo tập dữ liệu cho mô hình (với look_back đã được định nghĩa trước đó)
X_train, y_train = create_dataset(train_data_scaled, look_back)
X_validation, y_validation = create_dataset(validation_data_scaled, look_back)
X_test, y_test = create_dataset(test_data_scaled, look_back)

# Định dạng lại cho LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_validation = np.reshape(X_validation, (X_validation.shape[0], X_validation.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

print("Dữ liệu đã được chuẩn hóa và chia thành các tập train, validation, test.")
print("Kích thước tập train:", X_train.shape, y_train.shape)
print("Kích thước tập validation:", X_validation.shape, y_validation.shape)
print("Kích thước tập test:", X_test.shape, y_test.shape)

In [ ]:
# Xây dựng mô hình LSTM
model = Sequential()
model.add(LSTM(50, input_shape=(look_back, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
# Huấn luyện mô hình
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1, validation_data=(X_validation, y_validation))

In [ ]:
# Dự đoán
pred = model.predict(X_test)
# Khử chuẩn kết quả dự báo
pred = scaler.inverse_transform(pred)
print(pred)

In [ ]:
import matplotlib.pyplot as plt

# Lấy dữ liệu gốc cho tập kiểm tra (chỉ lấy phần tương ứng với các điểm dự báo)
original_test_data = scaler.inverse_transform(y_test)

# Vẽ đồ thị
plt.figure(figsize=(12, 6))
# Plot original test data
plt.plot(original_test_data, label='Chuỗi gốc (Test)', linestyle='-', linewidth=2, color='blue')
# Plot predicted test data
plt.plot(pred, label='Chuỗi dự báo (Test)', linestyle='-', linewidth=2, color='red')

plt.title('Đồ thị chuỗi thời gian gốc và chuỗi dự báo trên tập test')
plt.xlabel('Thời gian (trong tập test)')
plt.ylabel('Giá trị')
plt.legend()
plt.show()

In [ ]:
import pandas as pd

# Lấy dữ liệu gốc cho tập kiểm tra (đã khử chuẩn)
original_test_data_denormalized = scaler.inverse_transform(y_test)

# Tạo DataFrame
results_df = pd.DataFrame({
    'Original (Test)': original_test_data_denormalized.flatten(),
    'Predicted (Test)': pred.flatten()
})

print("DataFrame chứa dữ liệu gốc và dự báo trên tập kiểm tra:")
display(results_df)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

def smape(y_true, y_pred):
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    return np.mean(numerator / denominator) * 100

# Tính toán RMSE, MAE, MSE
rmse = np.sqrt(mean_squared_error(results_df['Original (Test)'], results_df['Predicted (Test)']))
mae = mean_absolute_error(results_df['Original (Test)'], results_df['Predicted (Test)'])
smape_value = smape(results_df['Original (Test)'], results_df['Predicted (Test)'])


print(f"RMSE trên tập kiểm tra: {rmse:.4f}")
print(f"MAE trên tập kiểm tra: {mae:.4f}")
print(f"SMAPE trên tập kiểm tra: {smape_value:.4f}%")

In [ ]:
cpi_data_reshaped = cpi_data.reshape(-1, 1)
print("Dữ liệu CPI đã được định hình lại:")
display(cpi_data_reshaped)